# Import

In [1]:
import os
import sys
sys.path.append(os.path.join(os.getcwd(),".."))

import cv2
import torch
import math
import glob
import scipy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import time
from sklearn.metrics import mean_squared_error as mse

import pytorch_kinematics as pk
from urdf_parser_py.urdf import URDF

# Helper Functions

In [2]:
def load_json(filename: str):
    # Construct the absolute path by joining the current directory and relative path
    absolute_path = os.path.join(os.getcwd(), '..', filename)
    # Load the JSON data
    with open(absolute_path, 'r') as file:
        json_data = json.load(file)    
    return json_data

def projectPoints(pts_3d, cam_mtx, ext_mtx, dist_coef, dtype=torch.float64,device="cuda"):
    # Transform to Homogeneous Coordinates
    pts_3d_homo = torch.cat((pts_3d, torch.ones(pts_3d.shape[0], 1, dtype=dtype, device=device)), dim=1)
    
    # Apply extrinsic transformation (rotation + translation)
    pts_3d_transformed = ext_mtx @ pts_3d_homo.T
    
    # Convert back from homogeneous to 2D coordinates
    pts_2d_prime = pts_3d_transformed[:2, :] / pts_3d_transformed[2, :]
    x_prime = pts_2d_prime[0]
    y_prime = pts_2d_prime[1]
    
    # Distortion Parameters Calculation
    k1, k2, p1, p2, k3 = dist_coef
    
    r2 = x_prime**2 + y_prime**2
    r4 = r2**2
    r6 = r2*r4
    a1 = 2*x_prime*y_prime
    a2 = r2 + 2*x_prime**2
    a3 = r2 + 2*y_prime**2
    
    # Radial distortion
    radial_distortion = 1 + k1*r2 + k2*r4 + k3*r6
    
    # Tangential distortion
    tangential_distortion_x = p1*a1 + p2*a2
    tangential_distortion_y = p1*a3 + p2*a1
    
    # Apply distortion
    x_double_prime = x_prime*radial_distortion + tangential_distortion_x
    y_double_prime = y_prime* radial_distortion + tangential_distortion_y
    
    # Intrinsic Camera Matrix
    fx = cam_mtx[0,0]
    cx = cam_mtx[0,2]
    fy = cam_mtx[1,1]
    cy = cam_mtx[1,2]
    
    u = x_double_prime*fx + cx
    v = y_double_prime*fy + cy
    
    px_pts = torch.stack((u, v), dim=1)

    return px_pts

def xml_to_str(robot):
    # To XML string with filtering
    temp_str = robot.to_xml_string()
    words = temp_str.split()
    words[5] = '>'
    urdf_str = ' '.join(words)
    return urdf_str

# Initialization

In [3]:
# Load Dataset CSV

data_dir = "final"
csv_file = os.path.join(os.getcwd(),"..","data", data_dir, "240909_grace_dataset.csv")
df = pd.read_csv(csv_file)
temp_df = df.iloc[:,:12]
print('Total:',len(temp_df))
temp_df

Total: 209731


,x_c_l,y_c_l,z_c_l,x_c_r,y_c_r,z_c_r,cmd_theta_lower_neck_pan,cmd_theta_lower_neck_tilt,cmd_theta_upper_neck_tilt,cmd_theta_left_eye_pan,cmd_theta_right_eye_pan,cmd_theta_eyes_tilt
0,-0.873669,-0.334702,0.738088,-1.166921,-0.352033,0.738508,-35,-10,-10,-14,-14,20
1,-0.821681,-0.331576,0.738000,-0.997318,-0.340332,0.738221,-35,-10,-10,-12,-12,20
2,-0.731501,-0.324165,0.738176,-0.868484,-0.332085,0.738360,-35,-10,-10,-10,-10,20
3,-0.672994,-0.320809,0.737795,-0.776648,-0.327031,0.737932,-35,-10,-10,-8,-8,20
4,-0.613507,-0.317492,0.737656,-0.690487,-0.321762,0.737752,-35,-10,-10,-6,-6,20
...,...,...,...,...,...,...,...,...,...,...,...,...
209726,0.792502,-0.267949,0.950136,0.716083,-0.253872,0.953027,35,30,40,4,4,-30
209727,0.871620,-0.284761,0.947121,0.792232,-0.271009,0.950140,35,30,40,6,6,-30
209728,0.958315,-0.302900,0.943909,0.875639,-0.289312,0.947054,35,30,40,8,8,-30
209729,1.054044,-0.323434,0.940457,0.967562,-0.310196,0.943755,35,30,40,10,10,-30


In [4]:
# Convert DF Degrees Column to Radians

data_df = temp_df.copy()
data_df['cmd_theta_lower_neck_pan'] = np.radians(temp_df['cmd_theta_lower_neck_pan'].values)
data_df['cmd_theta_lower_neck_tilt'] = np.radians(temp_df['cmd_theta_lower_neck_tilt'].values)
data_df['cmd_theta_upper_neck_tilt'] = np.radians(temp_df['cmd_theta_upper_neck_tilt'].values)
data_df['cmd_theta_left_eye_pan'] = np.radians(temp_df['cmd_theta_left_eye_pan'].values)
data_df['cmd_theta_right_eye_pan'] = np.radians(temp_df['cmd_theta_right_eye_pan'].values)
data_df['cmd_theta_eyes_tilt'] = np.radians(temp_df['cmd_theta_eyes_tilt'].values)
data_df

,x_c_l,y_c_l,z_c_l,x_c_r,y_c_r,z_c_r,cmd_theta_lower_neck_pan,cmd_theta_lower_neck_tilt,cmd_theta_upper_neck_tilt,cmd_theta_left_eye_pan,cmd_theta_right_eye_pan,cmd_theta_eyes_tilt
0,-0.873669,-0.334702,0.738088,-1.166921,-0.352033,0.738508,-0.610865,-0.174533,-0.174533,-0.244346,-0.244346,0.349066
1,-0.821681,-0.331576,0.738000,-0.997318,-0.340332,0.738221,-0.610865,-0.174533,-0.174533,-0.209440,-0.209440,0.349066
2,-0.731501,-0.324165,0.738176,-0.868484,-0.332085,0.738360,-0.610865,-0.174533,-0.174533,-0.174533,-0.174533,0.349066
3,-0.672994,-0.320809,0.737795,-0.776648,-0.327031,0.737932,-0.610865,-0.174533,-0.174533,-0.139626,-0.139626,0.349066
4,-0.613507,-0.317492,0.737656,-0.690487,-0.321762,0.737752,-0.610865,-0.174533,-0.174533,-0.104720,-0.104720,0.349066
...,...,...,...,...,...,...,...,...,...,...,...,...
209726,0.792502,-0.267949,0.950136,0.716083,-0.253872,0.953027,0.610865,0.523599,0.698132,0.069813,0.069813,-0.523599
209727,0.871620,-0.284761,0.947121,0.792232,-0.271009,0.950140,0.610865,0.523599,0.698132,0.104720,0.104720,-0.523599
209728,0.958315,-0.302900,0.943909,0.875639,-0.289312,0.947054,0.610865,0.523599,0.698132,0.139626,0.139626,-0.523599
209729,1.054044,-0.323434,0.940457,0.967562,-0.310196,0.943755,0.610865,0.523599,0.698132,0.174533,0.174533,-0.523599


In [5]:
# PyTorch Configs

device = "cuda" if torch.cuda.is_available() else "cpu"
dtype = torch.float64

In [6]:
# Load Configs
cam_mtxs = load_json('camera_mtx.json')
gaze_ctrs = load_json('calib_params.json')
urdf_path = os.path.join(os.getcwd(),"..","urdf","chest_grace.urdf")
robot = URDF.from_xml_file(urdf_path)

In [7]:
# Decision Variables Initial Value

var_dict= {
    'neck_pitch_polyfit_b0': 0.1,
    'neck_pitch_polyfit_b1': 0.1,
    'neck_pitch_polyfit_b2': 0.1,
    'neck_pitch_polyfit_b3': 0.1,

    'neck_yaw_polyfit_b0': 0.1,
    'neck_yaw_polyfit_b1': 0.1,
    'neck_yaw_polyfit_b2': 0.1,
    'neck_yaw_polyfit_b3': 0.1,

    'head_pitch_polyfit_b0': 0.1,
    'head_pitch_polyfit_b1': 0.1,
    'head_pitch_polyfit_b2': 0.1,
    'head_pitch_polyfit_b3': 0.1,

    'eyes_pitch_polyfit_b0': 0.1,
    'eyes_pitch_polyfit_b1': 0.1,
    'eyes_pitch_polyfit_b2': 0.1,
    'eyes_pitch_polyfit_b3': 0.1,

    'lefteye_yaw_polyfit_b0': 0.1,
    'lefteye_yaw_polyfit_b1': 0.1,
    'lefteye_yaw_polyfit_b2': 0.1,
    'lefteye_yaw_polyfit_b3': 0.1,

    'righteye_yaw_polyfit_b0': 0.1,
    'righteye_yaw_polyfit_b1': 0.1,
    'righteye_yaw_polyfit_b2': 0.1,
    'righteye_yaw_polyfit_b3': 0.1,

    'torso_origin_x': 0.12234,
    'torso_origin_y': -0.0325,
    'torso_origin_z': 0.05692,

    'neck_pitch_rot_r': 0,
    
    'head_pitch_origin_z': 0.13172,
    'head_pitch_rot_r': 0,
    'head_pitch_rot_y': 0,

    'eyes_pitch_origin_x': 0.08492,
    'eyes_pitch_origin_z': 0.05186,
    'eyes_pitch_rot_r': 0,
    'eyes_pitch_rot_y': 0,

    'lefteye_yaw_origin_y': 0.02895,
    
    'righteye_yaw_origin_y': -0.02895,

    'lefteye_cam_origin_x': 0.015,
    'lefteye_cam_rot_r': 0,
    'lefteye_cam_rot_p': 0,
    'lefteye_cam_rot_y': 0,

    'righteye_cam_origin_x': 0.015,
    'righteye_cam_rot_r': 0,
    'righteye_cam_rot_p': 0,
    'righteye_cam_rot_y': 0,
}

In [8]:
# Decision Variables 

var_list = list(var_dict.values())
var_list

[0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.1,
 0.12234,
 -0.0325,
 0.05692,
 0,
 0.13172,
 0,
 0,
 0.08492,
 0.05186,
 0,
 0,
 0.02895,
 -0.02895,
 0.015,
 0,
 0,
 0,
 0.015,
 0,
 0,
 0]

In [9]:
# Variable Names
var_names_list = list(var_dict.keys())
var_names_list

['neck_pitch_polyfit_b0',
 'neck_pitch_polyfit_b1',
 'neck_pitch_polyfit_b2',
 'neck_pitch_polyfit_b3',
 'neck_yaw_polyfit_b0',
 'neck_yaw_polyfit_b1',
 'neck_yaw_polyfit_b2',
 'neck_yaw_polyfit_b3',
 'head_pitch_polyfit_b0',
 'head_pitch_polyfit_b1',
 'head_pitch_polyfit_b2',
 'head_pitch_polyfit_b3',
 'eyes_pitch_polyfit_b0',
 'eyes_pitch_polyfit_b1',
 'eyes_pitch_polyfit_b2',
 'eyes_pitch_polyfit_b3',
 'lefteye_yaw_polyfit_b0',
 'lefteye_yaw_polyfit_b1',
 'lefteye_yaw_polyfit_b2',
 'lefteye_yaw_polyfit_b3',
 'righteye_yaw_polyfit_b0',
 'righteye_yaw_polyfit_b1',
 'righteye_yaw_polyfit_b2',
 'righteye_yaw_polyfit_b3',
 'torso_origin_x',
 'torso_origin_y',
 'torso_origin_z',
 'neck_pitch_rot_r',
 'head_pitch_origin_z',
 'head_pitch_rot_r',
 'head_pitch_rot_y',
 'eyes_pitch_origin_x',
 'eyes_pitch_origin_z',
 'eyes_pitch_rot_r',
 'eyes_pitch_rot_y',
 'lefteye_yaw_origin_y',
 'righteye_yaw_origin_y',
 'lefteye_cam_origin_x',
 'lefteye_cam_rot_r',
 'lefteye_cam_rot_p',
 'lefteye_cam_rot_

In [10]:
# Var Indexing
var_names_list.index('righteye_cam_rot_y')

44

In [11]:
# Index to Name Mapping
var2idx = dict(zip(list(range(len(var_names_list))),var_names_list))
var2idx

{0: 'neck_pitch_polyfit_b0',
 1: 'neck_pitch_polyfit_b1',
 2: 'neck_pitch_polyfit_b2',
 3: 'neck_pitch_polyfit_b3',
 4: 'neck_yaw_polyfit_b0',
 5: 'neck_yaw_polyfit_b1',
 6: 'neck_yaw_polyfit_b2',
 7: 'neck_yaw_polyfit_b3',
 8: 'head_pitch_polyfit_b0',
 9: 'head_pitch_polyfit_b1',
 10: 'head_pitch_polyfit_b2',
 11: 'head_pitch_polyfit_b3',
 12: 'eyes_pitch_polyfit_b0',
 13: 'eyes_pitch_polyfit_b1',
 14: 'eyes_pitch_polyfit_b2',
 15: 'eyes_pitch_polyfit_b3',
 16: 'lefteye_yaw_polyfit_b0',
 17: 'lefteye_yaw_polyfit_b1',
 18: 'lefteye_yaw_polyfit_b2',
 19: 'lefteye_yaw_polyfit_b3',
 20: 'righteye_yaw_polyfit_b0',
 21: 'righteye_yaw_polyfit_b1',
 22: 'righteye_yaw_polyfit_b2',
 23: 'righteye_yaw_polyfit_b3',
 24: 'torso_origin_x',
 25: 'torso_origin_y',
 26: 'torso_origin_z',
 27: 'neck_pitch_rot_r',
 28: 'head_pitch_origin_z',
 29: 'head_pitch_rot_r',
 30: 'head_pitch_rot_y',
 31: 'eyes_pitch_origin_x',
 32: 'eyes_pitch_origin_z',
 33: 'eyes_pitch_rot_r',
 34: 'eyes_pitch_rot_y',
 35: 'le

In [12]:
data_df['cmd_theta_lower_neck_pan'][0]

-0.6108652381980153

In [13]:
def objective_function(V):
    u_left_list = []
    v_left_list = []
    u_right_list = []
    v_right_list = []
    for i in range(len(data_df)):
        ### Joint-to-Motor Polynomial Models
        x = data_df['cmd_theta_lower_neck_tilt'][i]
        neck_pitch = V[0] + V[1]*x + V[2]*x**2 + V[3]*x**3
    
        x = data_df['cmd_theta_lower_neck_pan'][i]
        neck_yaw = V[4] + V[5]*x + V[6]*x**2 + V[7]*x**3
    
        x = data_df['cmd_theta_upper_neck_tilt'][i]
        head_pitch = V[8] + V[9]*x + V[10]*x**2 + V[11]*x**3
    
        x = data_df['cmd_theta_eyes_tilt'][i]
        eyes_pitch = V[12] + V[13]*x + V[14]*x**2 + V[15]*x**3
    
        x = data_df['cmd_theta_left_eye_pan'][i]
        lefteye_yaw = V[16] + V[17]*x + V[18]*x**2 + V[19]*x**3
    
        x = data_df['cmd_theta_right_eye_pan'][i]
        righteye_yaw = V[20] + V[21]*x + V[22]*x**2 + V[23]*x**3
    
        
        ### URDF
        # Variable Assignment
        for joint in robot.joints:
            if joint.name == 'torso':
                joint.origin.position[0] = V[24]
                joint.origin.position[1] = V[25]
                joint.origin.position[2] = V[26]
            elif joint.name == 'neck_pitch':
                joint.origin.rotation[0] = V[27]
            elif joint.name == 'head_pitch':
                joint.origin.position[2] = V[28]
                joint.origin.rotation[0] = V[29]
                joint.origin.rotation[2] = V[30]
            elif joint.name == 'eyes_pitch':
                joint.origin.position[0] = V[31]
                joint.origin.position[2] = V[32]
                joint.origin.rotation[0] = V[33]
                joint.origin.rotation[2] = V[34]
            elif joint.name == 'lefteye_yaw':
                joint.origin.position[1] = V[35]
            elif joint.name == 'righteye_yaw':
                joint.origin.position[1] = V[36]
            elif joint.name == 'lefteye_cam':
                joint.origin.position[0] = V[37]
                joint.origin.rotation[0] = V[38]
                joint.origin.rotation[1] = V[39]
                joint.origin.rotation[2] = V[40]
            elif joint.name == 'righteye_cam':
                joint.origin.position[0] = V[41]
                joint.origin.rotation[0] = V[42]
                joint.origin.rotation[1] = V[43]
                joint.origin.rotation[2] = V[44]
    
        # XML to String
        urdf_str = xml_to_str(robot)
    
        # Kinematic Chain
        chain = pk.build_chain_from_urdf(urdf_str)
    
        # Specifying Joint Angles (radians)
        joint_cmd = {
            'neck_pitch': neck_pitch,
            'neck_yaw': neck_yaw,
            'head_pitch': head_pitch,
            'eyes_pitch': eyes_pitch,
            'lefteye_yaw': lefteye_yaw,
            'righteye_yaw': righteye_yaw,
        }
    
        # Forward Kinematics
        ret = chain.forward_kinematics(joint_cmd)
    
        # Chest Camera OpenCV to World Orientation (LHR > RHR)
        T_opencv_world = torch.Tensor([[0.0, 0.0, 1.0, 0.0],
                                       [-1.0, 0.0, 0.0, 0.0],
                                       [0.0, -1.0, 0.0, 0.0],
                                       [0.0, 0.0, 0.0, 1.0]])
    
        # World Orientation to Chest Camera OpenCV (RHR > LHR )
        T_world_opencv = torch.Tensor([[0.0, -1.0, 0.0, 0.0],
                                       [0.0, 0.0, -1.0, 0.0],
                                       [1.0, 0.0, 0.0, 0.0],
                                       [0.0, 0.0, 0.0, 1.0]])
    
        # Realsense to Left Eye Camera (Pytorch) with Points
        T_clprime = (torch.linalg.inv(ret['realsense'].get_matrix()) @ ret['leftcamera'].get_matrix()).squeeze()
    
        # Realsense to Right Eye Camera (Pytorch) with Points
        T_crprime = (torch.linalg.inv(ret['realsense'].get_matrix()) @ ret['rightcamera'].get_matrix()).squeeze()
    
        # Extrinsic Matrix
        T_left_ext = T_world_opencv @ T_clprime @ T_opencv_world
        T_right_ext = T_world_opencv @ T_crprime @ T_opencv_world
    
    
        ### Camera Projection
        # Camera Parameters
        left_cam_mtx = torch.Tensor(cam_mtxs['left_eye']['camera_matrix'])
        left_dist_coef = torch.Tensor(cam_mtxs['left_eye']['distortion_coefficients']).squeeze()
        right_cam_mtx = torch.Tensor(cam_mtxs['right_eye']['camera_matrix'])
        right_dist_coef = torch.Tensor(cam_mtxs['right_eye']['distortion_coefficients']).squeeze()
    
        # Chest Camera OpenCV Orientation 3D Projection Points
        left_chest_cam_pt = torch.Tensor([[data_df['x_c_l'][i], data_df['y_c_l'][i], data_df['z_c_l'][i]]])
        right_chest_cam_pt = torch.Tensor([[data_df['x_c_r'][i], data_df['y_c_r'][i], data_df['z_c_r'][i]]])
    
        # Points Projection
        left_cam_px = projectPoints(left_chest_cam_pt, left_cam_mtx, T_left_ext, left_dist_coef).squeeze()
        right_cam_px = projectPoints(right_chest_cam_pt, right_cam_mtx, T_right_ext, right_dist_coef).squeeze()

        # Store the Pixel Projections
        u_left_list.append(left_cam_px[0].item())
        v_left_list.append(left_cam_px[1].item())
        u_right_list.append(left_cam_px[0].item())
        v_right_list.append(left_cam_px[1].item())


    # MSE Calculation
    true_u_left = len(data_df) * gaze_ctrs['left_eye']['x_center']
    true_v_left = len(data_df) * gaze_ctrs['left_eye']['y_center']
    true_u_right = len(data_df) * gaze_ctrs['right_eye']['x_center']
    true_v_right = len(data_df) * gaze_ctrs['right_eye']['y_center']

    loss = (mse(true_u_left,u_left_list) + mse(true_v_left,v_left_list) 
            + mse(true_u_right,u_right_list) + mse(true_v_right,u_right_list))
    
    # Return
    return loss   
    

In [14]:
# Single GPU Item Implementation

start = time.time()

V = var_list

i = 0

### Joint-to-Motor Polynomial Models
x = data_df['cmd_theta_lower_neck_tilt'][i]
neck_pitch = V[0] + V[1]*x + V[2]*x**2 + V[3]*x**3

x = data_df['cmd_theta_lower_neck_pan'][i]
neck_yaw = V[4] + V[5]*x + V[6]*x**2 + V[7]*x**3

x = data_df['cmd_theta_upper_neck_tilt'][i]
head_pitch = V[8] + V[9]*x + V[10]*x**2 + V[11]*x**3

x = data_df['cmd_theta_eyes_tilt'][i]
eyes_pitch = V[12] + V[13]*x + V[14]*x**2 + V[15]*x**3

x = data_df['cmd_theta_left_eye_pan'][i]
lefteye_yaw = V[16] + V[17]*x + V[18]*x**2 + V[19]*x**3

x = data_df['cmd_theta_right_eye_pan'][i]
righteye_yaw = V[20] + V[21]*x + V[22]*x**2 + V[23]*x**3


### URDF
# Variable Assignment
for joint in robot.joints:
    if joint.name == 'torso':
        joint.origin.position[0] = V[24]
        joint.origin.position[1] = V[25]
        joint.origin.position[2] = V[26]
    elif joint.name == 'neck_pitch':
        joint.origin.rotation[0] = V[27]
    elif joint.name == 'head_pitch':
        joint.origin.position[2] = V[28]
        joint.origin.rotation[0] = V[29]
        joint.origin.rotation[2] = V[30]
    elif joint.name == 'eyes_pitch':
        joint.origin.position[0] = V[31]
        joint.origin.position[2] = V[32]
        joint.origin.rotation[0] = V[33]
        joint.origin.rotation[2] = V[34]
    elif joint.name == 'lefteye_yaw':
        joint.origin.position[1] = V[35]
    elif joint.name == 'righteye_yaw':
        joint.origin.position[1] = V[36]
    elif joint.name == 'lefteye_cam':
        joint.origin.position[0] = V[37]
        joint.origin.rotation[0] = V[38]
        joint.origin.rotation[1] = V[39]
        joint.origin.rotation[2] = V[40]
    elif joint.name == 'righteye_cam':
        joint.origin.position[0] = V[41]
        joint.origin.rotation[0] = V[42]
        joint.origin.rotation[1] = V[43]
        joint.origin.rotation[2] = V[44]

# XML to String
urdf_str = xml_to_str(robot)

# Kinematic Chain
chain = pk.build_chain_from_urdf(urdf_str)
chain.to(dtype=dtype, device=device)

# Specifying Joint Angles (radians)
joint_cmd = {
    'neck_pitch': neck_pitch,
    'neck_yaw': neck_yaw,
    'head_pitch': head_pitch,
    'eyes_pitch': eyes_pitch,
    'lefteye_yaw': lefteye_yaw,
    'righteye_yaw': righteye_yaw,
}

# Forward Kinematics
ret = chain.forward_kinematics(joint_cmd)

# Chest Camera OpenCV to World Orientation (LHR > RHR)
T_opencv_world = torch.Tensor([[0.0, 0.0, 1.0, 0.0],
                               [-1.0, 0.0, 0.0, 0.0],
                               [0.0, -1.0, 0.0, 0.0],
                               [0.0, 0.0, 0.0, 1.0]]).to(dtype=dtype, device=device)

# World Orientation to Chest Camera OpenCV (RHR > LHR)
T_world_opencv = torch.Tensor([[0.0, -1.0, 0.0, 0.0],
                               [0.0, 0.0, -1.0, 0.0],
                               [1.0, 0.0, 0.0, 0.0],
                               [0.0, 0.0, 0.0, 1.0]]).to(dtype=dtype, device=device)

# Realsense to Left Eye Camera (Pytorch) with Points
T_clprime = (torch.linalg.inv(ret['realsense'].get_matrix()) @ ret['leftcamera'].get_matrix()).squeeze()

# Realsense to Right Eye Camera (Pytorch) with Points
T_crprime = (torch.linalg.inv(ret['realsense'].get_matrix()) @ ret['rightcamera'].get_matrix()).squeeze()

# Extrinsic Matrix
T_left_ext = T_world_opencv @ T_clprime @ T_opencv_world
T_right_ext = T_world_opencv @ T_crprime @ T_opencv_world


### Camera Projection
# Camera Parameters
left_cam_mtx = torch.Tensor(cam_mtxs['left_eye']['camera_matrix']).to(dtype=dtype, device=device)
left_dist_coef = torch.Tensor(cam_mtxs['left_eye']['distortion_coefficients']).squeeze().to(dtype=dtype, device=device)
right_cam_mtx = torch.Tensor(cam_mtxs['right_eye']['camera_matrix']).to(dtype=dtype, device=device)
right_dist_coef = torch.Tensor(cam_mtxs['right_eye']['distortion_coefficients']).squeeze().to(dtype=dtype, device=device)

# Chest Camera OpenCV Orientation 3D Projection Points
left_chest_cam_pt = torch.Tensor([[data_df['x_c_l'][i], data_df['y_c_l'][i], data_df['z_c_l'][i]]]).to(dtype=dtype, device=device)
right_chest_cam_pt = torch.Tensor([[data_df['x_c_r'][i], data_df['y_c_r'][i], data_df['z_c_r'][i]]]).to(dtype=dtype, device=device)

# Points Projection
left_cam_px = projectPoints(left_chest_cam_pt, left_cam_mtx, T_left_ext, left_dist_coef).squeeze()
right_cam_px = projectPoints(right_chest_cam_pt, right_cam_mtx, T_right_ext, right_dist_coef).squeeze()

end = time.time()
print('Elapsed Time: %.f ms' % (1000*(end-start)))

Elapsed Time: 207 ms


In [ ]:
# Specify Joint Angles (radians)
joint_cmd = {
    'neck_pitch': 0.0,
    'neck_yaw': 0.0,
    'head_pitch': math.radians(10),
    'eyes_pitch': 0.0,
    'lefteye_yaw': 0.0,
    'righteye_yaw': 0.0,
}

In [ ]:
# Testing the Function

#--- Input
pts_3d = torch.tensor([[1.0, 2.0, 5.0]])
cam_mtx = torch.tensor([[1000.0, 0, 320.0],
                        [0, 1000.0 , 240.0],
                        [0, 0, 1.0]])
ext_mtx = torch.tensor([[1.0, 0.0, 0.0, 0.0],
                        [0.0, 1.0, 0.0, 0.0],
                        [0.0, 0.0, 1.0, 0.0],
                        [0.0, 0.0, 0.0, 1.0]])
dist_coef = (0.1, -0.2, 0.01, 0.01, 0.0)

#--- Function
pytorch_pts = projectPoints(pts_3d, cam_mtx, ext_mtx, dist_coef)
pytorch_pts

In [ ]:
left_cam_mtx = torch.Tensor(cam_mtxs['left_eye']['camera_matrix'])
left_dist_coef = torch.Tensor(cam_mtxs['left_eye']['distortion_coefficients']).squeeze()
print(left_cam_mtx)
print(left_dist_coef)

In [ ]:
# Specify Joint Angles (radians)
joint_cmd = {
    'neck_pitch': 0.0,
    'neck_yaw': 0.0,
    'head_pitch': 0.0,
    'eyes_pitch': 0.0,
    'lefteye_yaw': 0.0,
    'righteye_yaw': 0.0,
}

In [ ]:
# Forward Kinematics
ret = chain.forward_kinematics(joint_cmd)
ret

In [ ]:
# Realsense to Left Eye Camera (Pytorch) with Points
T_clprime = (torch.linalg.inv(ret['realsense'].get_matrix()) @ ret['leftcamera'].get_matrix()).squeeze()

# Realsense to Right Eye Camera (Pytorch) with Points
T_crprime = (torch.linalg.inv(ret['realsense'].get_matrix()) @ ret['rightcamera'].get_matrix()).squeeze()

In [ ]:
# Chest Camera OpenCV to World Orientation (LHR > RHR)
T_opencv_world = torch.Tensor([[0.0, 0.0, 1.0, 0.0],
                               [-1.0, 0.0, 0.0, 0.0],
                               [0.0, -1.0, 0.0, 0.0],
                               [0.0, 0.0, 0.0, 1.0]])

# World Orientation to Chest Camera OpenCV (RHR > LHR )
T_world_opencv = torch.Tensor([[0.0, -1.0, 0.0, 0.0],
                               [0.0, 0.0, -1.0, 0.0],
                               [1.0, 0.0, 0.0, 0.0],
                               [0.0, 0.0, 0.0, 1.0]])

In [ ]:
T_ext = T_world_opencv @ T_clprime @ T_opencv_world
T_ext

In [ ]:
# Testing the Function

#--- Input
pts_3d = torch.tensor([[0.5, 0.5, 1.0]])

#--- Function
pytorch_pts = projectPoints(pts_3d, left_cam_mtx, T_ext, left_dist_coef)
pytorch_pts

# URDF

In [ ]:
# URDF Object Properties

for joint in robot.joints:
    print(joint.name)

In [ ]:
# Content Manipulation

for joint in robot.joints:
    if joint.name == 'head_pitch':
        print('Joint Object:',joint.origin)
        # Getting
        print("Single Attribute:",joint.origin.position)
        # Setting
        print("Attribute Type:",type(joint.origin.position))
        joint.origin.position = [joint.origin.position[0], joint.origin.position[1], joint.origin.position[2]]
        joint.origin.position[0] = joint.origin.position[0]
        print("Output:",joint.origin.position, joint.origin.rotation)
        print("Output:",joint.origin.position[0], joint.origin.rotation)

In [ ]:
# To XML string with filtering

temp_str = robot.to_xml_string()
words = temp_str.split()
words[5] = '>'
urdf_str = ' '.join(words)
print(urdf_str)

# Pytorch Kinematics

In [ ]:
chain = pk.build_chain_from_urdf(urdf_str)
chain.print_tree()

In [ ]:
print(chain.get_joint_parameter_names())

In [ ]:
# Specify Joint Angles (radians)
joint_cmd = {
    'neck_pitch': 0.0,
    'neck_yaw': 0.0,
    'head_pitch': 0.0,
    'eyes_pitch': 0.0,
    'lefteye_yaw': 0.0,
    'righteye_yaw': 0.0,
}

In [ ]:
# Forward Kinematics
ret = chain.forward_kinematics(joint_cmd)
ret

In [ ]:
# Forward Kinematics
ret = chain.forward_kinematics(joint_cmd)
ret

In [ ]:
ret['leftcamera'].get_matrix().numpy()

In [ ]:
ret['realsense'].get_matrix().numpy()

In [ ]:
# Realsense to Left Eye Camera (Numpy)
(np.linalg.inv(ret['realsense'].get_matrix().numpy()) @ ret['leftcamera'].get_matrix().numpy()).squeeze()

In [ ]:
# Realsense to Left Eye Camera (Pytorch)
(torch.linalg.inv(ret['realsense'].get_matrix()) @ ret['leftcamera'].get_matrix()).squeeze()

In [ ]:
# Realsense to Left Eye Camera (Pytorch) with Points
x,y,z = 1,1,1
torch.linalg.inv(ret['realsense'].get_matrix()) @ ret['leftcamera'].get_matrix() @ torch.Tensor([x,y,z,1])

In [ ]:
chain.get_frame_indices("leftcamera","rightcamera","realsense")

In [ ]:
torch.tensor(np.arange(1,11))

In [ ]:
# Forward Kinematics (Specific=frame indices)
ret = chain.forward_kinematics(joint_cmd, torch.tensor(np.arange(1,11)))
ret

In [ ]:
ret['leftcamera'].get_matrix()

In [ ]:
ret['rightcamera'].get_matrix()

In [ ]:
left_serialchain = pk.SerialChain(chain, "leftcamera", "torso")
left_serialchain.print_tree()

In [ ]:
# Forward Kinematics (Specific=frame indices)
ret = left_serialchain.forward_kinematics(torch.Tensor([0.0,0.0,0.0,0.0,0.0]), end_only=False)
ret

In [ ]:
ret['leftcamera'].get_matrix()

# Eye Projection

## Function Prototype

In [ ]:
#--- Input
pts_3d = torch.tensor([[1.0, 2.0, 5.0],
                      [2.0, 3.0, 8.0],
                      [3.0, 1.0, 10.0]])
cam_mtx = torch.tensor([[1000.0, 0, 320.0],
                        [0, 1000.0 , 240.0],
                        [0, 0, 1.0]])
ext_mtx = torch.tensor([[1.0, 0.0, 0.0, 0.0],
                        [0.0, 1.0, 0.0, 0.0],
                        [0.0, 0.0, 1.0, 0.0],
                        [0.0, 0.0, 0.0, 1.0]])
dist_coef = (0.1, -0.2, 0.01, 0.01, 0.0)

In [ ]:
# Transform to Homogeneous Coordinates
pts_3d_homo = torch.cat((pts_3d, torch.ones(pts_3d.shape[0], 1)), dim=1)
print(pts_3d_homo)
print(pts_3d_homo.T)

In [ ]:
# Apply extrinsic transformation (rotation + translation)
pts_3d_transformed = ext_mtx @ pts_3d_homo.T
pts_3d_transformed

In [ ]:
# Convert back from homogeneous to 2D coordinates
pts_2d_prime = pts_3d_transformed[:2, :] / pts_3d_transformed[2, :]
x_prime = pts_2d_prime[0]
y_prime = pts_2d_prime[1]
print(pts_2d_prime)
print(x_prime)
print(y_prime)

In [ ]:
# Distortion Parameters Calculation
k1, k2, p1, p2, k3 = dist_coef

r2 = x_prime**2 + y_prime**2
r4 = r2**2
r6 = r2*r4
a1 = 2*x_prime*y_prime
a2 = r2 + 2*x_prime**2
a3 = r2 + 2*y_prime**2

# Radial distortion
radial_distortion = 1 + k1*r2 + k2*r4 + k3*r6

# Tangential distortion
tangential_distortion_x = p1*a1 + p2*a2
tangential_distortion_y = p1*a3 + p2*a1

# Apply distortion
x_double_prime = x_prime*radial_distortion + tangential_distortion_x
y_double_prime = y_prime* radial_distortion + tangential_distortion_y

print(x_double_prime)
print(y_double_prime)

In [ ]:
# Intrinsic Camera Matrix
fx = cam_mtx[0,0]
cx = cam_mtx[0,2]
fy = cam_mtx[1,1]
cy = cam_mtx[1,2]

u = x_double_prime*fx + cx
v = y_double_prime*fy + cy

px_pts = torch.stack((u, v), dim=1)

print(u)
print(v)
print(px_pts)

## Function

In [ ]:
def projectPoints(pts_3d, cam_mtx, ext_mtx, dist_coef):
    # Transform to Homogeneous Coordinates
    pts_3d_homo = torch.cat((pts_3d, torch.ones(pts_3d.shape[0], 1)), dim=1)
    
    # Apply extrinsic transformation (rotation + translation)
    pts_3d_transformed = ext_mtx @ pts_3d_homo.T
    
    # Convert back from homogeneous to 2D coordinates
    pts_2d_prime = pts_3d_transformed[:2, :] / pts_3d_transformed[2, :]
    x_prime = pts_2d_prime[0]
    y_prime = pts_2d_prime[1]
    
    # Distortion Parameters Calculation
    k1, k2, p1, p2, k3 = dist_coef
    
    r2 = x_prime**2 + y_prime**2
    r4 = r2**2
    r6 = r2*r4
    a1 = 2*x_prime*y_prime
    a2 = r2 + 2*x_prime**2
    a3 = r2 + 2*y_prime**2
    
    # Radial distortion
    radial_distortion = 1 + k1*r2 + k2*r4 + k3*r6
    
    # Tangential distortion
    tangential_distortion_x = p1*a1 + p2*a2
    tangential_distortion_y = p1*a3 + p2*a1
    
    # Apply distortion
    x_double_prime = x_prime*radial_distortion + tangential_distortion_x
    y_double_prime = y_prime* radial_distortion + tangential_distortion_y
    
    # Intrinsic Camera Matrix
    fx = cam_mtx[0,0]
    cx = cam_mtx[0,2]
    fy = cam_mtx[1,1]
    cy = cam_mtx[1,2]
    
    u = x_double_prime*fx + cx
    v = y_double_prime*fy + cy
    
    px_pts = torch.stack((u, v), dim=1)

    return px_pts

In [ ]:
# Testing the Function

#--- Input
pts_3d = torch.tensor([[1.0, 2.0, 5.0],
                      [2.0, 3.0, 8.0],
                      [3.0, 1.0, 10.0]])
cam_mtx = torch.tensor([[1000.0, 0, 320.0],
                        [0, 1000.0 , 240.0],
                        [0, 0, 1.0]])
ext_mtx = torch.tensor([[1.0, 0.0, 0.0, 0.0],
                        [0.0, 1.0, 0.0, 0.0],
                        [0.0, 0.0, 1.0, 0.0],
                        [0.0, 0.0, 0.0, 1.0]])
dist_coef = (0.1, -0.2, 0.01, 0.01, 0.0)

#--- Function
pytorch_pts = projectPoints(pts_3d, cam_mtx, ext_mtx, dist_coef)
pytorch_pts

## OpenCV Comparison

In [ ]:
# Input

#--- Input
pts_3d = np.array([[1.0, 2.0, 5.0],
                      [2.0, 3.0, 8.0],
                      [3.0, 1.0, 10.0]])
cam_mtx = np.array([[1000.0, 0, 320.0],
                        [0, 1000.0 , 240.0],
                        [0, 0, 1.0]])
rvec = np.array([0.0,0.0,0.0]).reshape(-1,1)
tvec = np.array([0.0,0.0,0.0]).reshape(-1,1)
dist_coef = np.array([0.1, -0.2, 0.01, 0.01, 0.0])

In [ ]:
# OpenCV Function

opencv_pts,_ = cv2.projectPoints(pts_3d, rvec, tvec, cam_mtx, dist_coef)
opencv_pts

In [ ]:
# Comparison

opencv_pts.squeeze() - pytorch_pts.numpy()